In [45]:
import os
import xmlschema
import glob
import re
import difflib
import copy
import pprint


In [4]:
mae_schema = xmlschema.XMLSchema('./mae_schema.xsd')

gs_dict = mae_schema.to_dict("./gold_anots/1_37.txt.gs.xml")
app_dict = mae_schema.to_dict("./app_anots/1_37.txt.xml")

gs_text = gs_dict['TEXT']
app_text = app_dict['TEXT']

print(gs_dict.keys())


dict_keys(['TEXT', 'TAGS'])


In [16]:
type(gs_dict)


dict

In [6]:
# version 1 - cleanup some whitespaces and use difflib
# version 2 - cleanup all whitespaces

def clean_up_text(text):
    text = re.sub(re.compile('\.'), '', text)
    text = re.sub(re.compile('\n\n+'), '\n', text)
    text = re.sub(re.compile('\n'), ' ', text)
    text = re.sub(re.compile('\s\s+'), ' ', text)
#     text = re.sub(re.compile('\s'), '', text)
#     text = re.sub(re.compile('\.|,'), '', text)
    return text.strip()
    
gs_clean = clean_up_text(gs_text)
app_clean = clean_up_text(app_text)

In [8]:
cherry_pick = 420
print(gs_clean[cherry_pick:cherry_pick+200])
print(app_clean[cherry_pick:cherry_pick+200])

print(len(gs_clean), len(app_clean))


owadzone w ramach należnej mu pełnej autonomii powinny służyć osiąganiu tych celów, Senat Uniwersytetu Warszawskiego stanowi, co następuje: DZIAŁ I UNIWERSYTET WARSZAWSKI POSTANOWIENIA OGÓLNE § 1 Sied
owadzone w ramach należnej mu pełnej autonomii powinny służyć osiąganiu tych celów, Senat Uniwersytetu Warszawskiego stanowi, co następuje : DZIAŁ I UNIWERSYTET WARSZAWSKI POSTANOWIENIA OGÓLNE § 1 Sie
30073 30085


In [38]:
for filepath in glob.glob('./app_anots/*'):
    filename = filepath.split('\\')[-1]
    if not filename.startswith('37'):
        continue
    gs_dict = mae_schema.to_dict(f"./gold_anots/{filename.replace('txt','txt.gs')}")
    app_dict = mae_schema.to_dict(f"./app_anots/{filename}")
    print(filename, len(clean_up_text(gs_dict['TEXT'])), 'vs', len(clean_up_text(app_dict['TEXT'])), 'gs vs app')
    gs_clean_text = clean_up_text(gs_dict['TEXT'])
    app_clean_text = clean_up_text(app_dict['TEXT'])
    for i, s in enumerate(difflib.ndiff(gs_clean_text, app_clean_text)):
        if s[0] != ' ':
            print(i, s)
            print(gs_clean_text[i - 50: i+50])
            print(app_clean_text[i - 50: i+50])
    print(clean_up_text(gs_dict['TEXT'])[:100])
    print(clean_up_text(app_dict['TEXT'])[:100])
    break
    # assert(clean_up_text(gs_dict['TEXT']) == clean_up_text(app_dict['TEXT']))

    

37_38.txt.xml 2049 vs 2052 gs vs app
1248 +  
uczycielami akademickimi, stanowi nie mniej niż 25% składu Senatu, przy czym liczba pracowników nieb
uczycielami akademickimi, stanowi nie mniej niż 25 % składu Senatu, przy czym liczba pracowników nie
1507 +  
 najmniejszą liczbą całkowitą nie mniejszą niż 20% składu Senatu Liczbę przedstawicieli studentów i 
t najmniejszą liczbą całkowitą nie mniejszą niż 20 % składu Senatu Liczbę przedstawicieli studentów 
1939 +  
estor, osoba reprezentująca system biblioteczno-informacyjny, oraz przedstawiciele związków zawodowy
kwestor, osoba reprezentująca system biblioteczno- informacyjny, oraz przedstawiciele związków zawod
§ 37 Senat 1 Członkiem Senatu może być osoba spełniająca warunki wskazane w art 29 ust 4 Ustawy 2 Ka
§ 37 Senat 1 Członkiem Senatu może być osoba spełniająca warunki wskazane w art 29 ust 4 Ustawy 2 Ka


In [11]:
def fix_file(filename):
    gs_dict = mae_schema.to_dict(f"./gold_anots/{filename}.txt.gs.xml")
    app_dict = mae_schema.to_dict(f"./app_anots/{filename}.txt.xml")
    
    
fix_file('37_38')
    

In [83]:
def parse_tags(file_dict):
    tags = file_dict['TAGS']
    copy_tags = copy.deepcopy(tags)
    for tag in copy_tags:
        for annotation_tag in copy_tags[tag]:
            annotation_tag['start'] = int(annotation_tag['@spans'].split('~')[0])
            annotation_tag['end'] = int(annotation_tag['@spans'].split('~')[1])
    return copy_tags

def update_span(tags, index, diff):
    change = 1 if diff == '+' else -1
    for tag in tags:
        for annotation_tag in tags[tag]:
            print(annotation_tag)
            if annotation_tag['start'] >= index:
                annotation_tag['start'] = annotation_tag['start'] - change
                print(annotation_tag, 'start')
            if annotation_tag['end'] >= index:
                annotation_tag['end'] = annotation_tag['end'] - change
                print(annotation_tag, 'end')
                
        
    

gs_tags = parse_tags(gs_dict)
# pprint.pprint(gs_tags['oBject'])
update_span(gs_tags, 350, '+')
# pprint.pprint(gs_tags['oBject'])


{'@id': 'S0', '@spans': '14~16', '@text': '1.', 'start': 14, 'end': 16}
{'@id': 'S1', '@spans': '106~108', '@text': '2.', 'start': 106, 'end': 108}
{'@id': 'S2', '@spans': '198~199', '@text': '.', 'start': 198, 'end': 199}
{'@id': 'S3', '@spans': '395~397', '@text': '1.', 'start': 395, 'end': 397}
{'@id': 'S3', '@spans': '395~397', '@text': '1.', 'start': 394, 'end': 397} start
{'@id': 'S3', '@spans': '395~397', '@text': '1.', 'start': 394, 'end': 396} end
{'@id': 'S4', '@spans': '586~588', '@text': '2.', 'start': 586, 'end': 588}
{'@id': 'S4', '@spans': '586~588', '@text': '2.', 'start': 585, 'end': 588} start
{'@id': 'S4', '@spans': '586~588', '@text': '2.', 'start': 585, 'end': 587} end
{'@id': 'S5', '@spans': '784~785', '@text': '.', 'start': 784, 'end': 785}
{'@id': 'S5', '@spans': '784~785', '@text': '.', 'start': 783, 'end': 785} start
{'@id': 'S5', '@spans': '784~785', '@text': '.', 'start': 783, 'end': 784} end
{'@id': 'S6', '@spans': '911~913', '@text': '3.', 'start': 911, 'e

In [40]:
clean_up_text(gs_dict['TEXT'])[1248]

'%'

In [27]:
gs_dict = mae_schema.to_dict("./gold_anots/1_37.txt.gs.xml")
app_dict = mae_schema.to_dict("./app_anots/1_37.txt.xml")